In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [33]:
vocab_50000 = open("/Users/yishh/Downloads/cs224u_chatbot/movie_50000.txt").readlines()

In [134]:
cleaned_vocab = Counter()
for ix,v in enumerate(vocab_50000):
    curr_v = v.replace("\n","")
    cleaned_vocab[curr_v] = ix

In [159]:
cleaned_vocab

Counter({'': 48,
         'fawn': 22507,
         'nunnery': 26253,
         'sonja': 22726,
         'tilton': 46823,
         'woods': 2174,
         'clotted': 47104,
         'spiders': 12288,
         'hanging': 1663,
         'francesca': 11937,
         'sevens': 17960,
         'disobeying': 27295,
         'canes': 39967,
         'cordette': 31569,
         'canez': 41675,
         'scold': 6103,
         'brodeigh': 49336,
         'originality': 39285,
         'mutinies': 39286,
         'hermann': 11224,
         'statute': 19195,
         'oshizu': 23372,
         'spider-': 39968,
         'grenadiers': 19338,
         'wood-': 42583,
         'stipulate': 36460,
         'langrad': 48184,
         'appropriation': 37793,
         'rawhide': 32760,
         'politician': 8437,
         'iisteners': 44225,
         'bringing': 1609,
         'thattaboy': 31675,
         'neum\xc3\xbciler': 36282,
         'caramels': 16489,
         'wooden': 5080,
         'doodle-': 91

In [112]:
sep = "+++$+++"
cornell_file = open("/Users/yishh/Desktop/projects/CS224U_NeuralConvo/movie_lines.txt")
movie_lines = []

conversation_memo = {}

for l in cornell_file.readlines():
    split_l = l.split(sep)
    
    person_a = split_l[1].strip()
    person_b = split_l[2].strip()
    
    pair_key = "*".join(sorted([person_a,person_b]))
    
    curr_l = split_l[-1]
    curr_l = curr_l.replace("\n","")[1::]
    movie_lines.append(curr_l)
    
    if pair_key in conversation_memo:
        conversation_memo[pair_key].append(curr_l)
    else:
        conversation_memo[pair_key] = [curr_l]
    

In [113]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
for key, val in conversation_memo.items():
    accum = []
    for s in val:
        s = unicode(s, errors='replace')
        tkn = tknzr.tokenize(s.lower())
        accum += tkn
    accum = Counter(accum)
    conversation_memo[key] = accum

In [142]:
#number of pairs of characters exchanges (documents)
num_docs = len(conversation_memo)
doc_word_M = np.zeros((num_docs,len(vocab_50000)))

In [143]:
index = 0
for key, val in conversation_memo.items():
    doc_word_M[index,]
    for word,counts in val.items():
        if word in cleaned_vocab:
            doc_word_M[index, cleaned_vocab[word]] = counts
    index += 1

In [139]:
np.save("doc_word_M",doc_word_M)

In [183]:
doc_word_M.shape

(9035, 50000)

In [184]:
def obtainIDF(words_doc_matrix):
    pass

doc_word_M.T


array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [ 219.,    2.,   11., ...,   12.,   26.,   12.],
       [  58.,    0.,    0., ...,   16.,   37.,    6.],
       ..., 
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [147]:
def tfidf(mat, rownames=None):
    """TF-IDF 
    
    Parameters
    ----------
    mat : 2d np.array
       The matrix to operate on.
       
    rownames : list of str or None
        Not used; it's an argument only for consistency with other methods 
        defined here.
        
    Returns
    -------
    (np.array, list of str)    
       The first member is the TF-IDF-transformed version of `mat`, and 
       the second member is `rownames` (unchanged).
    
    """
    colsums = np.sum(mat, axis=0)
    doccount = mat.shape[1]
    w = np.array([_tfidf_row_func(row, colsums, doccount) for row in mat])
#     w = np.array([_idf_row_func(row, colsums, doccount) for row in mat])
    return (w, rownames)

def idf(mat, rownames=None):
    """TF-IDF 
    
    Parameters
    ----------
    mat : 2d np.array
       The matrix to operate on.
       
    rownames : list of str or None
        Not used; it's an argument only for consistency with other methods 
        defined here.
        
    Returns
    -------
    (np.array, list of str)    
       The first member is the TF-IDF-transformed version of `mat`, and 
       the second member is `rownames` (unchanged).
    
    """
    colsums = np.sum(mat, axis=0)
    doccount = mat.shape[1]
    w = np.array([_idf_row_func(row, colsums, doccount) for row in mat])
    return (w, rownames)

def _tfidf_row_func(row, colsums, doccount):
    df = float(len([x for x in row if x > 0]))
    idf = 0.0
    # This ensures a defined IDF value >= 0.0:
    if df > 0.0 and df != doccount:
        idf = np.log(doccount / df)
    tfs = row/colsums
    return tfs * idf


def _idf_row_func(row, colsums, doccount):
    df = float(len([x for x in row if x > 0]))
    idf = 0.0
    # This ensures a defined IDF value >= 0.0:
    if df > 0.0 and df != doccount:
        idf = np.log(doccount / df)
#     tfs = row/colsums
    return idf

In [145]:
TFIDF = tfidf(doc_word_M, None)[0]

In [181]:
TFIDF[0][:100]

array([[             nan,   2.89387664e-03,   1.53195603e-03, ...,
                     nan,              nan,   0.00000000e+00],
       [             nan,   4.57099109e-05,   0.00000000e+00, ...,
                     nan,              nan,   0.00000000e+00],
       [             nan,   2.19118158e-04,   0.00000000e+00, ...,
                     nan,              nan,   0.00000000e+00],
       ..., 
       [             nan,   3.66949442e-04,   3.84204132e-04, ...,
                     nan,              nan,   0.00000000e+00],
       [             nan,   6.51304328e-05,   4.33955421e-05, ...,
                     nan,              nan,   0.00000000e+00],
       [             nan,   6.77626461e-05,   0.00000000e+00, ...,
                     nan,              nan,   0.00000000e+00]])

In [149]:
IDF = idf(doc_word_M.T,None)[0]

In [152]:
np.save("IDF",IDF)

In [185]:
IDF[:100]

array([ 0.        ,  0.01798027,  0.07752789,  0.12141441,  0.19733126,
        0.15547973,  0.1310836 ,  0.73553838,  0.20926684,  0.21748719,
        0.        ,  0.33207682,  0.        ,  0.62770978,  0.37397931,
        0.40801401,  0.50990371,  0.38286682,  0.38108299,  0.52475731,
        0.44808728,  0.43193295,  0.73185004,  0.62605244,  0.51008802,
        0.51193304,  0.68093648,  0.57225762,  0.60637564,  0.56931961,
        0.57402458,  4.12525458,  0.57796236,  0.        ,  0.64085826,
        0.70125969,  0.62501799,  0.64972094,  0.        ,  0.64359308,
        0.69347928,  0.        ,  0.62688077,  0.84064232,  0.77199157,
        0.61309596,  0.86689464,  0.92072779,  0.        ,  0.7840399 ,
        0.86926775,  1.5757024 ,  0.78938751,  0.95695133,  0.74575203,
        0.90884755,  1.28721808,  0.78476744,  1.09717447,  0.96767141,
        0.79207208,  0.85147555,  0.83501427,  0.93754433,  1.22466127,
        0.        ,  0.74598537,  0.7767935 ,  1.2220282 ,  0.87

Now that we have created the IDF matrix, calculate the TF-IDF score for every term

In [221]:
tknzr = TweetTokenizer()
def get_idf(tgt):
#     tkn = Counter(tknzr.tokenize(tgt))
    tkn = Counter(tgt.split(" ")[:-1])
    score = 0.0
    for key,val in tkn.items():
        score += 2 * IDF[int(key)]
#         print score, IDF[int(key)], key
#     print score
#     print score
    return score

In [222]:
get_idf("47 3 33 6 2386 2 ")

14.830351834284011

In [223]:
#read in the file
f_tgt_dev = open("/Users/yishh/Downloads/cs224u_chatbot/rearank_target_dev.txt").readlines()
f_tgt_valid = open("/Users/yishh/Downloads/cs224u_chatbot/rearank_target_valid.txt").readlines()


In [224]:
def calc_idf_for_file(f,outfile):
    with open(outfile,"w") as output:
        for line in f:
            output.write(str(get_idf(line)) + "\n")

In [225]:
calc_idf_for_file(f_tgt_dev, "tfidf_dev.txt")

In [226]:
calc_idf_for_file(f_tgt_valid, "tfidf_valid.txt")